In [ ]:
# write a few test documents with a sentence for each. They should all be very different
from hodja.search import Document, FAISS
from hodja.search.embeddings.openai import OpenAIEmbeddings
from hodja.tools.search_tools import SearchTool

test_documents = [
    Document(text="This is a test document.", id=1),
    Document(text="A man once asked for a glass of water.", id=2),
    Document(text="The man was thirsty.", id=3),
    Document(text="This is another test document.", id=4),
]

embeddings = OpenAIEmbeddings()
db = FAISS(embeddings=embeddings)

db.add(test_documents)

search_tool = SearchTool(db)

In [2]:
search_tool

Search: Search for documents based on a query. Returns a list of documents that best match the query.

In [9]:
from hodja.agents.openai import OpenAIAgent

In [1]:
"""Classes for interacting with OpenAI's API."""
import os
from hodja.agents.base import Agent
import openai
from collections import defaultdict

# create default dict of context sizes
CONTEXT_SIZES = defaultdict(lambda: 2048)
_defaults = {
    "text-davinci-003": 4097,
    "text-curie-001": 2048,
    "text-babbage-001": 2048,
    "text-ada-001": 2048,
    "code-davinci-002": 8000,
    "code-cushman-001": 2048,
}
for k, v in _defaults.items():
    CONTEXT_SIZES[k] = v

class OpenAIAgent(Agent):
    
    def __init__(
        self, 
        name='OpenAI Agent',
        api_key=os.getenv('OPENAI_API_KEY'),
        engine='text-davinci-003',
        temperature=0.0,
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=['\n\n',],
        streaming=False
        ):
        super().__init__(name=name)
        self.api_key = api_key
        self.engine = engine
        self.context_size = CONTEXT_SIZES[self.engine]
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.top_p = top_p
        self.frequency_penalty = frequency_penalty
        self.presence_penalty = presence_penalty
        self.stop = stop
        self.streaming = streaming

    def prepare_prompt_string(self, prompt, state, tools):
        """Formats the prompts with information from state"""
        return prompt.format(**state, tools=tools)

    def __call__(self, prompt, state, tools=[]):
        prompt_string = self.prepare_prompt_string(prompt, state, tools)
        if len(prompt_string) + self.max_tokens > self.context_size:
            raise ValueError(
                f"Prompt length ({len(prompt_string)}) + max_tokens ({self.max_tokens}) "
                f"exceeds maximum context size ({self.context_size})."
            )
        results = openai.Completion.create(
            prompt=prompt_string,
            engine=self.engine,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            top_p=self.top_p,
            frequency_penalty=self.frequency_penalty,
            presence_penalty=self.presence_penalty,
            stop=self.stop,
            stream=self.streaming
        )
        output = {
            'prompt': prompt,
            'completion': results.choices[0].text,
        }
        return output

agent = OpenAIAgent()

In [36]:
prompt = """You are Assistant. Your job is to follow these instructions.

INSTRUCTIONS
Please repeat the following text inside the ``` block back exactly as it is written:

```
user_input: {user_input}
tools: {tools}
```
END OF INSTRUCTIONS

You can now start your work.

Assistant Response:
"""

tools = ["Search", "Math"]

In [37]:
agent(prompt, {"user_input": "This is a test document."}, tools)

{'prompt': 'You are Assistant. Your job is to follow these instructions.\n\nINSTRUCTIONS\nPlease repeat the following text inside the ``` block back exactly as it is written:\n\n```\nuser_input: {user_input}\ntools: {tools}\n```\nEND OF INSTRUCTIONS\n\nYou can now start your work.\n\nAssistant Response:\n',
 'completion': "```\nuser_input: This is a test document.\ntools: ['Search', 'Math']\n```"}

In [2]:
from hodja import Chain
from hodja.links.react import ReACTLink
from hodja.tools.math import MathTool, FibonacciTool
tools = [MathTool(), FibonacciTool()]
links = [ReACTLink(tools=tools)]
chain = Chain("ReACT", links=links)
chain.run("What is the weather in Denver?", debug=True)

You are a polite, thoughtful, and resourceful general purpose AI. 

Tools - You can use the following tools. The tool name is listed first and a description is listed after the colon.
* Math: Evaluate a math expression. Note: only numbers and basic operators (e.g. +-*/) are allowed.
* Fibonacci: Returns the nth Fibonacci number

To employ a tool, use following format:
    Action: <tool name>[<tool input>]

Note: You MUST use the tool's name (as provided above) in order to use a tool.
    
Instructions: 
    1. Think about the problem and how to solve it.
    2. Perform an action. You can use one of the tools or return a final response with `RETURN[<final answer>]
    3. Observe the result of your action. This will be the output from any tool you used, or the final answer if you returned a final answer.
    4. Repeat steps 1-3 until you have a final answer in the observation.

Example 1 (no tool needed):
    ```
    User Input: What is captial of France?
    Tools Available: ()
    Thou

'I need to use a Weather tool to get the weather in Denver.'

In [3]:
fibonacci = FibonacciTool()
print(fibonacci.run(10))

89


In [3]:
def _fib(n):
    if n <= 1:
        return n
    else:
        return _fib(n-1) + _fib(n-2)
    
_fib(10)

55

In [8]:
6765-8848

-2083